In [32]:
import os
import fsspec
from PIL import Image as PILImage

from darknet.py import ImageClassifier

In [37]:
github_opts= dict(org="pjreddie", repo="darknet", sha="master")
filecache_opts = dict(cache_storage=f"{os.environ['HOME']}/.cache/darknet.py")

In [47]:
# Load the ImageNet 1k labels/metadata
with fsspec.open("filecache::github://data/imagenet.shortnames.list",
                 filecache=filecache_opts, github=github_opts) as f:
    labels = [line.decode().rstrip() for line in f.readlines()[:1000]]

In [48]:
config_of = fsspec.open("filecache::github://cfg/darknet53_448.cfg",
                     filecache=filecache_opts, github=github_opts)
weights_of = fsspec.open("filecache::https://pjreddie.com/media/files/darknet53_448.weights",
                      filecache=filecache_opts)
with config_of as config:
    with weights_of as weights:
        # Load the Classifier
        n = ImageClassifier(labels=labels,
                            config_file=config.name,
                            weights_file=weights.name)

In [49]:
dog_of = fsspec.open("filecache::github://data/dog.jpg",
                     filecache=filecache_opts, github=github_opts)
with dog_of as dog:
    res = n.classify(dog.name, top=5)
res

[('malamute', 0.98354006),
 ('Eskimo dog', 0.0042837244),
 ('Siberian husky', 0.0031863458),
 ('Tibetan mastiff', 0.0030448402),
 ('Great Pyrenees', 0.0022190544)]

In [50]:
with dog_of as dog:
    res = n.classify(PILImage.open(dog.name), top=5)
res

[('malamute', 0.98354006),
 ('Eskimo dog', 0.0042837244),
 ('Siberian husky', 0.0031863458),
 ('Tibetan mastiff', 0.0030448402),
 ('Great Pyrenees', 0.0022190544)]

In [52]:
try:
    import cv2
    # Note: Please note that cv2 nd-arrays are h*w*c ordered.
    with dog_of as dog:
        cv2_img = cv2.imread(dog.name)
    cv2_img = cv2.cvtColor(cv2_img, cv2.COLOR_BGR2RGB)
    n.classify(cv2_img, top=5)
except ModuleNotFoundError:
    pass

In [53]:
eagle_of = fsspec.open("filecache::github://data/eagle.jpg",
                     filecache=filecache_opts, github=github_opts)
with eagle_of as eagle:
    res = n.classify(eagle.name, top=5)
res


[('bald eagle', 0.55666465),
 ('vulture', 0.21876547),
 ('kite', 0.18937683),
 ('ruddy turnstone', 0.004589723),
 ('ruffed grouse', 0.0032499917)]

In [54]:
import requests
majesty_url="https://s3-us-west-2.amazonaws.com/static.music-man.com/website/images/instruments/instrument-77.png?1588624445"
img = PILImage.open(requests.get(majesty_url, stream=True).raw)
n.classify(img, top=5)

[('electric guitar', 0.98759043),
 ('acoustic guitar', 0.009553942),
 ('banjo', 0.0011607071),
 ('pick', 0.0007309786),
 ('stage', 0.00058993115)]